In [11]:
import sys
import boto3
import botocore
import os
import subprocess
import shutil
import errno
import re

In [12]:
directory = '/projects/ps-yeolab3/bay001/codebase/metadata/results_dir/john_doe_20180412/'

In [13]:
def copy_dir_aws(src, bucket):
    """
    Copies a directory from local to AWS

    :param src:
    :param dest:
    :param logger:
    :return:
    """
    if not bucket.startswith('s3://'):
        bucket = 's3://' + bucket

    if not src.endswith('/'):
        src = src + '/'

    upload_dir = os.path.dirname(src).split('/')[-1] + "/"

    cmd = 'aws s3 cp {} {} --recursive'.format(
        src,
        os.path.join(bucket, upload_dir)
    )
    try:
        print("Uploading to aws: {}".format(cmd))
        subprocess.check_call(cmd, shell=True)
        print("Done uploading {} to aws ({})".format(src, bucket))
    except Exception as e:
        print(e)

In [14]:
copy_dir_aws(
    directory, 'metadata-results'
)

Uploading to aws: aws s3 cp /projects/ps-yeolab3/bay001/codebase/metadata/results_dir/john_doe_20180412/ s3://metadata-results/john_doe_20180412/ --recursive
Done uploading /projects/ps-yeolab3/bay001/codebase/metadata/results_dir/john_doe_20180412/ to aws (s3://metadata-results)


In [ ]:
def copy_dir_aws(src, bucket):
    client = boto3.client('s3')
    destination = src.split('/')[-1]
    print("Destination: {}".format(destination))
    # enumerate local files recursively
    for root, dirs, files in os.walk(local_directory):

        for filename in files:

        # construct the full local path
        local_path = os.path.join(root, filename)

        # construct the full Dropbox path
        relative_path = os.path.relpath(local_path, local_directory)
        s3_path = os.path.join(destination, relative_path)

        # relative_path = os.path.relpath(os.path.join(root, filename))

        print('Searching "%s" in "%s"' % (s3_path, bucket))
        try:
            client.head_object(Bucket=bucket, Key=s3_path)
            print("Path found on S3! Skipping %s..." % s3_path)

            # try:
                # client.delete_object(Bucket=bucket, Key=s3_path)
            # except:
                # print "Unable to delete %s..." % s3_path
        except:
            print("Uploading {} to bucket {} with path {}".format(local_path, bucket, s3_path))
            # client.upload_file(local_path, bucket, s3_path)

In [ ]:
def copy_dir(src, dest):
    """ Copies a file or directory """
    
    copy_directory = os.path.join(dest, os.path.dirname(src).split('/')[-1] + "/")
    try:
        shutil.copytree(src, copy_directory)
    except OSError as exc:  # python >2.5
        if exc.errno == errno.ENOTDIR:
            shutil.copy(src, copy_directory)
        elif exc.errno == errno.EEXIST:
            print("Folder exists, will not overwrite.")
        else:
            raise

In [ ]:
src = '/projects/ps-yeolab3/bay001/codebase/metadata/work_dir/test_upload/'
dest = '/projects/ps-yeolab3/bay001/codebase/metadata/results_dir/'

In [ ]:
copy_dir(src, dest)

In [ ]:
dest = 's3://metadata-results'
if not src.endswith('/'):
    src = src + '/'
    
copy_directory = os.path.join(
    dest, os.path.dirname(src).split('/')[-1] + "/"
)

copy_directory

In [25]:
res = '/projects/ps-yeolab3/bay001/codebase/metadata/results_dir/john_doe_20180412/'

def remove_file(to_remove, logger):
    logger.info("Removing {}".format(to_remove))
    os.remove(to_remove)
    time.sleep(1)
    
def remove_unnecessary_intermediates(res, pipeline='dropseqtools'):
    print("Removing unnecessary intermediate files.")
    if pipeline == 'dropseqtools':
        suffixes_to_remove = [
            ".sam$",
            ".tagged([\d]+-[\d]+).bam$",
            '.tagged([\d]+-[\d]+).tagged([\d]+-[\d]+).bam$',
            '.tagged([\d]+-[\d]+).tagged([\d]+-[\d]+).filtered.trimmed_smart.bam$',
            '.tagged([\d]+-[\d]+).tagged([\d]+-[\d]+).filtered.trimmed_smart.polyA_filtered.bam$',
            '.tagged([\d]+-[\d]+).tagged([\d]+-[\d]+).filtered.trimmed_smart.polyA_filtered.STARAligned.out.bam$',
            '.tagged([\d]+-[\d]+).tagged([\d]+-[\d]+).filtered.trimmed_smart.polyA_filtered.STARUnmapped.out.mate1$',
            '.tagged([\d]+-[\d]+).tagged([\d]+-[\d]+).filtered.trimmed_smart.polyA_filtered.STARAligned.out.namesorted.bam$',
            '.tagged([\d]+-[\d]+).tagged([\d]+-[\d]+).filtered.trimmed_smart.polyA_filtered.STARAligned.out.namesorted.merged.bam$',
            '.tagged([\d]+-[\d]+).tagged([\d]+-[\d]+).filtered.trimmed_smart.polyA_filtered.STARAligned.out.namesorted.merged.TaggedGeneExon.bam$',
        ]
    else:
        suffixes_to_remove = []  # TODO: cellranger
    for suffix in suffixes_to_remove:
        for f in os.listdir(os.path.join(res, "results")):
            regex = '[\w\d]+' + suffix
            match = re.compile(regex).match
            if match(f):
                print("Removing: {} ({})".format(f, regex))
                # remove_file(to_remove=os.path.join(res, "results", f), logger=logger)
    print("Done removing unnecessary files.")

In [26]:
remove_unnecessary_intermediates(res)

Removing unnecessary intermediate files.
Removing: run_01_sample_01.sam ([\w\d]+.sam$)
Removing: run_01_sample_01.tagged1-12.bam ([\w\d]+.tagged([\d]+-[\d]+).bam$)
Removing: run_01_sample_01.tagged1-12.tagged13-20.bam ([\w\d]+.tagged([\d]+-[\d]+).tagged([\d]+-[\d]+).bam$)
Removing: run_01_sample_01.tagged1-12.tagged13-20.filtered.trimmed_smart.bam ([\w\d]+.tagged([\d]+-[\d]+).tagged([\d]+-[\d]+).filtered.trimmed_smart.bam$)
Removing: run_01_sample_01.tagged1-12.tagged13-20.filtered.trimmed_smart.polyA_filtered.bam ([\w\d]+.tagged([\d]+-[\d]+).tagged([\d]+-[\d]+).filtered.trimmed_smart.polyA_filtered.bam$)
Removing: run_01_sample_01.tagged1-12.tagged13-20.filtered.trimmed_smart.polyA_filtered.STARAligned.out.bam ([\w\d]+.tagged([\d]+-[\d]+).tagged([\d]+-[\d]+).filtered.trimmed_smart.polyA_filtered.STARAligned.out.bam$)
Removing: run_01_sample_01.tagged1-12.tagged13-20.filtered.trimmed_smart.polyA_filtered.STARUnmapped.out.mate1 ([\w\d]+.tagged([\d]+-[\d]+).tagged([\d]+-[\d]+).filtered.t

In [27]:
def copy_dir(src, dest):
    """
    Copies a file or directory locally.
    ie. copying path/to/src/ to otherpath/to/dest/ -> otherpath/to/dest/src

    :param src: basestring
    :param dest: basestring
        local destination
    :param logger: logging.Logger
        Logger object
    :return:
    """

    if not src.endswith('/'):
        src = src + '/'

    copy_directory = os.path.join(
        dest, os.path.dirname(src).split('/')[-1] + "/"
    )

    try:
        print("Copying {} to {}.".format(src, copy_directory))
        shutil.copytree(src, copy_directory)
        print("Done copying {} to {}".format(src, copy_directory))
    except OSError as exc:  # python >2.5
        if exc.errno == errno.ENOTDIR:
            print("Warning, {} not a directory.".format(src))
            shutil.copy(src, copy_directory)
        elif exc.errno == errno.EEXIST:
            print("Warning, folder {} exists, will not overwrite.".format(src))
        else:
            logger.error(exc)
            raise

In [28]:
src = '/projects/ps-yeolab3/bay001/codebase/metadata/results_dir/aichaim_cortical_organoids_20180424/'
dest = '/oasis/tscc/scratch/bay001/metadata/'
copy_dir(src, dest)

Copying /projects/ps-yeolab3/bay001/codebase/metadata/results_dir/aichaim_cortical_organoids_20180424/ to /oasis/tscc/scratch/bay001/metadata/aichaim_cortical_organoids_20180424/.
Done copying /projects/ps-yeolab3/bay001/codebase/metadata/results_dir/aichaim_cortical_organoids_20180424/ to /oasis/tscc/scratch/bay001/metadata/aichaim_cortical_organoids_20180424/


In [30]:
def copy_files(src, dest):
    """
    Copies a file locally.

    :param src: basestring
        source file
    :param dest: basestring
        destination
    :param logger: logging.Logger()
        Logger object.
    :return:
    """
    try:
        print("Copying {} to {}".format(src, dest))
        shutil.copy(src, dest)
        print("Done copying {} to {}".format(src, dest))
    except Exception as e:
        print(e)
        raise

In [32]:
src = '/oasis/tscc/scratch/bay001/metadata/aichaim_cortical_organoids_20180424/DROPSEQ_aichaim_cortical_organoids_20180424_INPUT.yaml'
dest = '/home/bay001/DROPSEQ_aichaim_cortical_organoids_20180424_INPUT.yaml'
copy_files(src, dest)

Copying /oasis/tscc/scratch/bay001/metadata/aichaim_cortical_organoids_20180424/DROPSEQ_aichaim_cortical_organoids_20180424_INPUT.yaml to /home/bay001/DROPSEQ_aichaim_cortical_organoids_20180424_INPUT.yaml
Done copying /oasis/tscc/scratch/bay001/metadata/aichaim_cortical_organoids_20180424/DROPSEQ_aichaim_cortical_organoids_20180424_INPUT.yaml to /home/bay001/DROPSEQ_aichaim_cortical_organoids_20180424_INPUT.yaml
